# **Evaluation**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import nltk
import pickle as pkl
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools,time
import sys, os
from collections import OrderedDict,Counter
from copy import deepcopy
from time import time
import matplotlib.pyplot as plt
import pickle as pkl
import sys, getopt
import re
import gensim

##### Load 20 News Groups Dataset(Saved)




In [0]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [0]:
newsgroups_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

remove headers footers quotes

In [0]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train_maintext = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'))
newsgroups_test_maintext = fetch_20newsgroups(subset='test',
                                      remove=('headers', 'footers', 'quotes'))
print(newsgroups_train_maintext.data[0])
print(newsgroups_test_maintext.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.


In [0]:
print('\n'.join(newsgroups_train.target_names))

#### Tokenize Dataset (Saved)

In [0]:
import spacy
import string

tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation
# lowercase and remove punctuation
def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text.lower() for token in tokens if (token.text not in punctuations)]

def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens



In [0]:
token_dataset_train, all_tokens_train = tokenize_dataset(newsgroups_train.data)
pkl.dump(token_dataset_train, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train.p", "wb"))
pkl.dump(all_tokens_train, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_train.p", "wb"))

In [0]:
token_dataset_test, all_tokens_test = tokenize_dataset(newsgroups_test.data)
pkl.dump(token_dataset_test, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test.p", "wb"))
pkl.dump(all_tokens_test, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_test.p", "wb"))

=========================================================  
remove header footers and quotes

In [0]:
token_dataset_train, all_tokens_train = tokenize_dataset(newsgroups_train_maintext.data)
pkl.dump(token_dataset_train, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_maintext_train.p", "wb"))
pkl.dump(all_tokens_train, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_maintext_train.p", "wb"))

In [0]:
token_dataset_test, all_tokens_test = tokenize_dataset(newsgroups_test_maintext.data)
pkl.dump(token_dataset_test, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_maintext_test.p", "wb"))
pkl.dump(all_tokens_test, open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_maintext_test.p", "wb"))

#### Preprocess and Clean Dataset (Saved)

###### load tokenized dataset

In [0]:
#with headers footers
token_dataset_train = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train.p", "rb"))
all_tokens_train = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_train.p", "rb"))


In [0]:
#without headers footers
token_dataset_train = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_maintext_train.p", "rb"))
all_tokens_train = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_maintext_train.p", "rb"))


###### preprocess funcs

In [0]:
import nltk
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools,time
import sys, os
from collections import OrderedDict,Counter
from copy import deepcopy
from time import time
import matplotlib.pyplot as plt
import pickle as pkl
import sys, getopt
import re
import gensim

from nltk.stem import WordNetLemmatizer 
wnl = WordNetLemmatizer() 
from nltk.stem import PorterStemmer
porter = PorterStemmer()

import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
words = set(nltk.corpus.words.words())
stops = set(stopwords.words('english'))

In [0]:

def process_text(texts, lemmatization = None, with_bigram=False, min_length = None):
    """
    Function to process texts. Following are the steps we take:
    
    1. Stopword Removal(nltk)
    2. Lemmatization (not stem since stemming can reduce the interpretability).
    3. Collocation detection. 
    
    Parameters:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    # stopword removal using NLTK's english stopwords dataset.
    texts = [[word for word in line if word not in stops and word.isalpha() ] for line in texts] 
    

    # Lemmatization is generally better than stemming in the case of topic modeling 
    if lemmatization:
        # texts = [[lemmatizer.lemmatize(word) for word in line] for line in texts] 

        texts = [[wnl.lemmatize(i,j[0].lower()) if j[0].lower() in lemmatization else wnl.lemmatize(i) for i,j in pos_tag(line)] for line in texts]

   

    # Bigram collocation detection (frequently co-occuring tokens) using gensim's Phrases. can even try trigram collocation detection.
    if with_bigram:
        bigram = gensim.models.Phrases(texts)
        texts = [bigram[line] for line in texts]

    if min_length:
        texts = [[word for word in line if len(word)> min_length] for line in texts]

    return texts


###### 1) stopwords, bigram

In [0]:
data_train = process_text(token_dataset_train, 
                          with_bigram=True)

data_idx = [0,10,11,20,34,3456,6643]
for idx in data_idx:
  print('original:\n', token_dataset_train[idx])
  print('cleaned:\n',data_train[idx])


original:
 ['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', '\n', 'the', 'other', 'day', 'it', 'was', 'a', '2-door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/', '\n', 'early', '70s', 'it', 'was', 'called', 'a', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', '\n', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', '\n', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', '\n', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', '\n', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'e', 'mail']
cleaned:
 ['wondering_anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sports_car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'bo

###### 2) stopwords, lemmatize, bigram


In [0]:
data_train_lem = process_text(token_dataset_train, 
                          lemmatization = ['n'], #['a','n','v']
                          with_bigram=True)

data_idx = [0,10,11,20,34,3456,6643]
for idx in data_idx:
  print('original:\n', token_dataset_train[idx])
  print('cleaned:\n',data_train_lem[idx])


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


original:
 ['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', '\n', 'the', 'other', 'day', 'it', 'was', 'a', '2-door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/', '\n', 'early', '70s', 'it', 'was', 'called', 'a', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', '\n', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', '\n', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', '\n', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', '\n', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'e', 'mail']
cleaned:
 ['wondering_anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sport_car', 'looked', 'late', 'early', 'called', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body

In [0]:
#also lemmatize a v
data_train_lem = process_text(token_dataset_train, 
                          lemmatization = ['a','n','v'],
                          with_bigram=True)

data_idx = [0,10,11,20,34,3456,6643]
for idx in data_idx:
  print('original:\n', token_dataset_train[idx])
  print('cleaned:\n',data_train_lem[idx])


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


original:
 ['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', '\n', 'the', 'other', 'day', 'it', 'was', 'a', '2-door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/', '\n', 'early', '70s', 'it', 'was', 'called', 'a', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', '\n', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', '\n', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', '\n', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', '\n', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'e', 'mail']
cleaned:
 ['wonder_anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sport_car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'kno

###### 3) stopwords, lemmatize, bigram, min_length > 1

In [0]:
#Bigran and remove nltk stopwords
data_train_lem_2 = process_text(token_dataset_train, 
                          lemmatization = ['a','n','v'],
                          with_bigram=True,
                          min_length = 1)

data_idx = [0,10,11,20,34,3456,6643]
for idx in data_idx:
  print('original:\n', token_dataset_train[idx])
  print('cleaned:\n',data_train_lem_2[idx])


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


original:
 ['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', '\n', 'the', 'other', 'day', 'it', 'was', 'a', '2-door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/', '\n', 'early', '70s', 'it', 'was', 'called', 'a', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', '\n', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', '\n', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', '\n', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', '\n', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'e', 'mail']
cleaned:
 ['wonder_anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sport_car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'kno

###### save cleaned tokens dataset

In [0]:
pkl.dump(data_train, open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train_bi.p','wb'))

In [0]:
#stopwords, lemmatize a n v, bigram, min_length > 1
pkl.dump(data_train_lem_2, open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train_lem_2.p','wb'))

In [0]:
#stopwords, bigram
pkl.dump(data_train, open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train_main_bi.p','wb'))

###### same process for test set

In [0]:
## Test Set
token_dataset_test = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test.p", "rb"))
all_tokens_test = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_test.p", "rb"))

data_test = process_text(token_dataset_test, 
                          with_bigram=True,
                          filter_stopword_spacy = False,
                          remove_non_eng = False,
                          with_lemmatize = False, 
                          with_stem = False)

pkl.dump(data_test, open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test_bi.p','wb'))

In [0]:
#test set without headers, footers, quotes
## Test Set stopwords, lemmatize a n v, bigram, min_length > 1
token_dataset_test = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_maintext_test.p", "rb"))
all_tokens_test = pkl.load(open("/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/all_tokens_20newsgroups_maintext_test.p", "rb"))

data_test = process_text(token_dataset_test, 
                          with_bigram=True)

pkl.dump(data_test, open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test_main_bi.p','wb'))

#### Load Preprocessed Clean Tokenized Dataset

In [0]:
#with header...
# data_train = pkl.load(open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train_bi.p','rb'))
# data_test = pkl.load(open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test_bi.p','rb'))

In [0]:
#without headers
# lem a n v min_length >1 
data_train = pkl.load(open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train_lem_2.p','rb'))
data_test = pkl.load(open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test_lem_2.p','rb'))
print(data_train[0])
print(data_test[0])

['wonder_anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sport_car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatever', 'info', 'funky', 'look', 'car', 'please_e', 'mail']
['little', 'confused', 'model', 'bonnevilles', 'heard', 'le', 'se', 'lse', 'sse', 'ssei', 'could_someone', 'tell_difference', 'far', 'feature', 'performance', 'also', 'curious_know', 'book', 'value', 'prefereably', 'model', 'much_le', 'book', 'value', 'usually', 'get', 'word', 'much', 'demand', 'time', 'year', 'heard', 'mid', 'spring', 'early', 'summer', 'best', 'time', 'buy']


In [0]:
#without headers
data_train = pkl.load(open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_train_main_bi.p','rb'))
data_test = pkl.load(open('/content/drive/My Drive/GradAcademics/2019Fall/Capstone/data/tokenized_20newsgroups_test_main_bi.p','rb'))
print(data_train[0])
print(data_test[0])

['wondering_anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sports_car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please_e', 'mail']
['little', 'confused', 'models', 'bonnevilles', 'heard', 'le', 'se', 'lse', 'sse', 'ssei', 'could_someone', 'tell', 'differences', 'far', 'features', 'performance', 'also', 'curious', 'know', 'book', 'value', 'prefereably', 'model', 'much_less', 'book', 'value', 'usually', 'get', 'words', 'much', 'demand', 'time', 'year', 'heard', 'mid', 'spring', 'early', 'summer', 'best', 'time', 'buy']


#### vocabulary prep

##### vocab

In [0]:
dictionary = gensim.corpora.Dictionary(data_train)

In [0]:
# print 10 words in vocab dict
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 addition
1 anyone
2 body
3 bricklin
4 called
5 car
6 could
7 day
8 doors
9 early
10 engine


In [0]:
len(dictionary)

62880

In [0]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 30% of all documents
'''
dictionary.filter_extremes(no_below=10, no_above=0.1, keep_n = 8000) 


In [0]:
len(dictionary)

8000

##### bow vectorize

In [0]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''

bow_corpus_train = [dictionary.doc2bow(doc) for doc in data_train]
bow_corpus_test =[dictionary.doc2bow(doc) for doc in data_test]

In [0]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 10
bow_doc_x = bow_corpus_train[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

## LDA

#### Train

In [0]:
# LDA multicore 
%%time
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
lda_model =  gensim.models.LdaMulticore(bow_corpus_train, 
                                   num_topics = 20, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

CPU times: user 1min 3s, sys: 11.5 s, total: 1min 15s
Wall time: 1min 45s


In [0]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    # print(topic)
    print("\n")

Topic: 0 
Words: 0.010*"government" + 0.008*"state" + 0.007*"president" + 0.006*"israel" + 0.005*"country" + 0.004*"israeli" + 0.004*"american" + 0.004*"war" + 0.004*"force" + 0.003*"turkish"


Topic: 1 
Words: 0.019*"god" + 0.015*"jesus" + 0.007*"church" + 0.007*"believe" + 0.006*"word" + 0.005*"bible" + 0.005*"point" + 0.005*"love" + 0.005*"sin" + 0.005*"mean"


Topic: 2 
Words: 0.034*"game" + 0.015*"win" + 0.012*"la" + 0.012*"score" + 0.011*"hit" + 0.010*"player" + 0.010*"van" + 0.009*"cal" + 0.009*"team" + 0.009*"pitch"


Topic: 3 
Words: 0.019*"image" + 0.015*"program" + 0.014*"color" + 0.011*"display" + 0.011*"window" + 0.007*"package" + 0.007*"thanks" + 0.007*"file" + 0.006*"graphic" + 0.006*"software"


Topic: 4 
Words: 0.012*"book" + 0.008*"do" + 0.008*"learn" + 0.007*"job" + 0.006*"new" + 0.006*"font" + 0.006*"build" + 0.005*"moon" + 0.005*"lot" + 0.005*"language"


Topic: 5 
Words: 0.046*"file" + 0.045*"x_x" + 0.017*"program" + 0.016*"entry" + 0.010*"line" + 0.009*"name" + 0

In [0]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    # print("Topic: {} \nWords: {}".format(idx, topic ))
    print(topic)
    print("\n")


0.048*"car" + 0.010*"tire" + 0.009*"model" + 0.007*"dealer" + 0.007*"anyone" + 0.006*"thanks" + 0.006*"owner" + 0.005*"bmw" + 0.005*"brake" + 0.005*"trade"


0.013*"application" + 0.013*"widget" + 0.012*"window" + 0.010*"motif" + 0.010*"set" + 0.009*"server" + 0.008*"display" + 0.008*"run" + 0.008*"code" + 0.008*"program"


0.007*"water" + 0.006*"etc" + 0.005*"book" + 0.005*"university" + 0.004*"post" + 0.004*"anyone" + 0.004*"question" + 0.004*"new" + 0.004*"heat" + 0.004*"metal"


0.020*"la" + 0.018*"van" + 0.016*"bos" + 0.016*"det" + 0.015*"cal" + 0.014*"pit" + 0.014*"tor" + 0.013*"min" + 0.013*"nj" + 0.012*"mon"


0.015*"sale" + 0.013*"sell" + 0.009*"price" + 0.007*"cd" + 0.007*"new" + 0.007*"ask" + 0.007*"interested" + 0.007*"include" + 0.006*"line" + 0.006*"printer"


0.040*"post" + 0.014*"article" + 0.014*"drug" + 0.009*"read" + 0.009*"group" + 0.007*"anyone" + 0.007*"newsgroup" + 0.006*"dod" + 0.006*"edge" + 0.005*"please"


0.008*"really" + 0.007*"point" + 0.006*"better" + 0.0

#### Topic Coherence

In [0]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model,
                                    texts = data_train, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)


Coherence Score: 0.4743163706982658


In [0]:
coherence_model_lda_test = CoherenceModel(model=lda_model,
                                    texts = data_test, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda_test = coherence_model_lda_test.get_coherence()
print('\nCoherence Score on test:', coherence_lda_test)


Coherence Score on test: nan


/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:317: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [0]:
np.isnan(coherence_lda_test)

True

In [0]:
from gensim.models import CoherenceModel
coherence_lda_test = np.nan
while np.isnan(coherence_lda_test):
  lda_model =  gensim.models.LdaMulticore(bow_corpus_train, 
                                    num_topics = 20, 
                                    id2word = dictionary,                                    
                                    passes = 10,
                                    workers = 2)
  coherence_model_lda = CoherenceModel(model=lda_model,
                                    texts = data_train, 
                                    dictionary = dictionary,
                                    coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score:', coherence_lda)

  coherence_model_lda_test = CoherenceModel(model=lda_model,
                                      texts = data_test, 
                                      dictionary = dictionary,
                                      coherence='c_v')
  coherence_lda_test = coherence_model_lda_test.get_coherence()
  print('\nCoherence Score on test:', coherence_lda_test)


Coherence Score: 0.5436908306531877

Coherence Score on test: 0.4879053489457356


===========================================  
yesterday

In [0]:
#vocab 6,000
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model,
                                    texts = data_train, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)


Coherence Score: 0.5670714144282154


In [0]:
coherence_model_lda_test = CoherenceModel(model=lda_model,
                                    texts = data_test, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda_test = coherence_model_lda_test.get_coherence()
print('\nCoherence Score on test:', coherence_lda_test)


Coherence Score on test: nan


In [0]:
#vocab 10,000
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model,
                                    texts = data_train, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)


Coherence Score: 0.4869315584837187


In [0]:

coherence_model_lda_test = CoherenceModel(model=lda_model,
                                    texts = data_test, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda_test = coherence_model_lda_test.get_coherence()
print('\nCoherence Score on test:', coherence_lda_test)

/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))



Coherence Score on test: nan


====================================================

In [0]:
#vocab 10,000
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model,
                                    texts = data_train, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)


Coherence Score: 0.47576536775224626


In [0]:
from gensim.models import CoherenceModel
coherence_model_lda_test = CoherenceModel(model=lda_model,
                                    texts = data_test, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_lda_test = coherence_model_lda_test.get_coherence()
print('\nCoherence Score on test:', coherence_lda_test)


Coherence Score on test: 0.44858534521467136


======================================================

#### Perplexity

In [0]:
import warnings
warnings.simplefilter('ignore')
perplexity = lda_model.log_perplexity(bow_corpus_train)
print('lda perplexity:', perplexity)

lda perplexity: -8.016822289837787


## HDP

In [0]:
hdp_model.evaluate_test_corpus(bow_corpus_test)

-3845594.7109833304

=========================================================

In [0]:
%%time
hdp_model = gensim.models.HdpModel(bow_corpus_train, dictionary)

CPU times: user 53.5 s, sys: 34.4 s, total: 1min 27s
Wall time: 46.6 s


In [0]:
hdp_model.print_topics()

[(0,
  '0.004*system + 0.004*file + 0.004*program + 0.003*new + 0.003*include + 0.003*run + 0.002*information + 0.002*support + 0.002*number + 0.002*year'),
 (1,
  '0.005*god + 0.003*believe + 0.003*point + 0.003*mean + 0.003*tell + 0.002*jesus + 0.002*car + 0.002*really + 0.002*still + 0.002*start'),
 (2,
  '0.005*include + 0.004*file + 0.004*system + 0.004*program + 0.003*available + 0.003*run + 0.003*version + 0.003*machine + 0.003*support + 0.003*software'),
 (3,
  '0.031*x_x + 0.006*program + 0.005*file + 0.004*line + 0.003*write + 0.003*read + 0.003*return + 0.003*open + 0.002*year + 0.002*include'),
 (4,
  '0.002*file + 0.002*vote + 0.002*image + 0.002*include + 0.002*name + 0.001*system + 0.001*mean + 0.001*post + 0.001*word + 0.001*new'),
 (5,
  '0.004*det + 0.004*pit + 0.004*van + 0.004*tor + 0.004*nj + 0.004*nyr + 0.004*la + 0.004*chi + 0.004*que + 0.004*nyi'),
 (6,
  '0.002*car + 0.002*drive + 0.002*year + 0.001*card + 0.001*insurance + 0.001*chipsets + 0.001*run + 0.001*ra

==================================================

In [0]:
coherence_model_hdp = CoherenceModel(model=hdp_model,
                                    texts = data_train, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_hdp = coherence_model_hdp.get_coherence()
print('\nCoherence Score:', coherence_hdp)


Coherence Score: 0.6399517430396299


In [2]:
coherence_model_hdp_test = CoherenceModel(model=hdp_model,
                                    texts = data_test, 
                                    dictionary = dictionary,
                                    coherence='c_v')
coherence_hdp_test = coherence_model_hdp_test.get_coherence()
print('\nCoherence Score:', coherence_hdp_test)


Coherence Score: 0.506


## CTM

## VAE

In [0]:
!pip install --upgrade gensim

In [0]:
# !pip install --upgrade gensim
special_tokens = {'pad': 0, 'unk':1}
dictionary.patch_with_special_tokens(special_tokens)


In [0]:
UNK_IDX = 1
token2id = dictionary.token2id
vocab_size = len(dictionary)
def token2index_dataset(tokens_data, token2id):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

x_train = token2index_dataset(data_train, token2id)
x_train = np.array([np.array(document) for document in x_train])


In [0]:
def onehot(data, min_length):
    return np.bincount(data, minlength=min_length)

x_train = np.array([onehot(doc.astype('int'),vocab_size) for doc in x_train if np.sum(doc)!=0])
n_samples_tr = x_train.shape[0]
docs_tr = x_train

In [0]:
def make_network(layer1=100,layer2=100,num_topics=20,batch_size=200,learning_rate=0.01):
    tf.reset_default_graph()
    network_architecture = \
        dict(n_hidden_recog_1=layer1, # 1st layer encoder neurons
             n_hidden_recog_2=layer2, # 2nd layer encoder neurons
             n_hidden_gener_1=x_train.shape[1], # 1st layer decoder neurons
             n_input=x_train.shape[1], # MNIST data input (img shape: 28*28)
             n_z=num_topics)  # dimensionality of latent space

    return network_architecture,batch_size,learning_rate

In [0]:
x_val = token2index_dataset(data_test, token2id)
x_val = np.array([np.array(document) for document in x_val])
x_val = np.array([onehot(doc.astype('int'),vocab_size) for doc in x_val if np.sum(doc)!=0])
n_samples_te = x_val.shape[0]
docs_te = x_val

In [0]:
slim = tf.contrib.slim

tf.reset_default_graph()

class VAE(object):
    """
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """


    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus,
                 learning_rate=0.01, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct

        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print('Initial Learning Rate:', self.learning_rate)

        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]], name='input')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

        self.h_dim = (network_architecture["n_z"]) # had a float before
        self.a = 1*np.ones((1 , self.h_dim)).astype(np.float32)                         # a    = 1
        self.prior_mean = tf.constant((np.log(self.a).T-np.mean(np.log(self.a),1)).T)          # prior_mean  = 0
        self.prior_var = tf.constant(  ( ( (1.0/self.a)*( 1 - (2.0/self.h_dim) ) ).T +       # prior_var = 0.99 + 0.005 = 0.995
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a,1) ).T  )
        self.prior_logvar = tf.log(self.prior_var)
        self.means = []

        self._create_network()
        with tf.name_scope('cost'):
            self._create_loss_optimizer()

        init = tf.initialize_all_variables()

        self.sess = tf.InteractiveSession()
        self.sess.run(init)

    def _create_network(self):
        """
        steps:
        1. initialize weights
        2. build recognition network
        3. build reconstruction network
        """
        n_z = self.network_architecture['n_z']
        n_hidden_gener_1 = self.network_architecture['n_hidden_gener_1']
        en1 = slim.layers.linear(self.x, self.network_architecture['n_hidden_recog_1'], scope='FC_en1')
        en1 = tf.nn.softplus(en1, name='softplus1')
        en2 = slim.layers.linear(en1,    self.network_architecture['n_hidden_recog_2'], scope='FC_en2')
        en2 = tf.nn.softplus(en2, name='softplus2')
        en2_do = slim.layers.dropout(en2, self.keep_prob, scope='en2_dropped')
        self.posterior_mean   = slim.layers.linear(en2_do, self.network_architecture['n_z'], scope='FC_mean')
        self.posterior_logvar = slim.layers.linear(en2_do, self.network_architecture['n_z'], scope='FC_logvar')
        self.posterior_mean   = slim.layers.batch_norm(self.posterior_mean, scope='BN_mean')
        self.posterior_logvar = slim.layers.batch_norm(self.posterior_logvar, scope='BN_logvar')
        
        with tf.name_scope('z_scope'):
            eps = tf.random_normal((self.batch_size, n_z), 0, 1,                            # take noise
                                   dtype=tf.float32)
            self.z = tf.add(self.posterior_mean,
                            tf.multiply(tf.sqrt(tf.exp(self.posterior_logvar)), eps))         # reparameterization z
            self.posterior_var = tf.exp(self.posterior_logvar) 

        self.p = slim.layers.softmax(self.z)
        p_do = slim.layers.dropout(self.p, self.keep_prob, scope='p_dropped')               # dropout(softmax(z))
        decoded = slim.layers.linear(p_do, n_hidden_gener_1, scope='FC_decoder')

        self.x_reconstr_mean = tf.nn.softmax(slim.layers.batch_norm(decoded, scope='BN_decoder'))                    # softmax(bn(50->1995))

        print(self.x_reconstr_mean)

    def _create_loss_optimizer(self):

        tensor = self.x * tf.log(self.x_reconstr_mean+1e-10)                                                   # prevent log(0)
        indices = [i for i in range(1,tensor.shape[1])] # exclude 'pad', include 'unk'
        result = tf.gather(tensor, indices, axis=1)
        NL = -tf.reduce_sum(result, 1)
        # NL = -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean+1e-10), 1)     # cross entropy on categorical:- sum(ylog(p))

        var_division    = self.posterior_var  / self.prior_var
        diff            = self.posterior_mean - self.prior_mean
        diff_term       = diff * diff / self.prior_var
        logvar_division = self.prior_logvar - self.posterior_logvar
        KLD = 0.5 * (tf.reduce_sum(var_division + diff_term + logvar_division, 1) - self.h_dim )

        self.cost = tf.reduce_mean(NL + KLD)
 
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.99,epsilon=0.01).minimize(self.cost)

    def partial_fit(self, X):

        #if hasattr(self, 'decoder_weight'):
            #decoder_weight = self.decoder_weight
        #else:
        decoder_weight = [v for v in tf.global_variables() if v.name=='FC_decoder/weights:0'][0]
        opt, cost,emb,p = self.sess.run((self.optimizer, self.cost, decoder_weight, self.p),feed_dict={self.x: X,self.keep_prob: .8})
        # print(self.sess.run((self.p),feed_dict={self.x: X,self.keep_prob: .8}))
        return cost,emb,p

    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost = self.sess.run((self.cost),feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0})
        return cost
    def topic_prop(self, X):
        """heta_ is the topic proportion vector. Apply softmax transformation to it before use.
        """
        theta_ = self.sess.run((self.z),feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0})
        return theta_

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
def make_network(layer1=100,layer2=100,num_topics=20,batch_size=200,learning_rate=0.01):
    tf.reset_default_graph()
    network_architecture = \
        dict(n_hidden_recog_1=layer1, # 1st layer encoder neurons
             n_hidden_recog_2=layer2, # 2nd layer encoder neurons
             n_hidden_gener_1=x_train.shape[1], # 1st layer decoder neurons
             n_input=x_train.shape[1], # MNIST data input (img shape: 28*28)
             n_z=num_topics)  # dimensionality of latent space

    return network_architecture,batch_size,learning_rate
    
def create_minibatch(data,batch_size):
    rng = np.random.RandomState(10)

    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]

In [0]:
def train(network_architecture, minibatches, type='prodlda',learning_rate=0.01,
          batch_size=200, training_epochs=100, display_step=5):
    tf.reset_default_graph()
    vae = VAE(network_architecture,transfer_fct=tf.nn.softplus,
                learning_rate=learning_rate, batch_size=batch_size)
    writer = tf.summary.FileWriter('logs', tf.get_default_graph())
    emb=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = next(minibatches)
            # Fit training using batch data
            cost,emb,p = vae.partial_fit(batch_xs)
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size

            if np.isnan(avg_cost):
                print(epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape)
                print('Encountered NaN, stopping training. Please check the learning_rate settings and the momentum.')
                # return vae,emb
                sys.exit()

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost))
            print_ppl(vae)
    return vae,emb

In [0]:
def print_top_words(beta, feature_names, n_top_words=10):
    print('---------------Printing the Topics------------------')
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))
    print('---------------End of Topics------------------')

In [0]:
def print_ppl(model):
    cost=[]
    for doc in docs_te:
        doc = doc.astype('float32')
        n_d = np.sum(doc[1:]) # count non-pad
        c=model.test(doc)
        if n_d==0:
            continue
        else:
            cost.append(c/n_d)
    print('The approximated perplexity for test set is: ',(np.exp(np.mean(np.array(cost)))))

In [0]:
network_architecture,batch_size,learning_rate=make_network(layer1=200,layer2=100,num_topics=20,
                                                           batch_size=256,learning_rate=0.01)
network_architecture

{'n_hidden_gener_1': 8001,
 'n_hidden_recog_1': 200,
 'n_hidden_recog_2': 100,
 'n_input': 8001,
 'n_z': 20}

In [0]:
%%time
minibatches = create_minibatch(docs_tr.astype('float32'),batch_size=batch_size)
vae,emb = train(network_architecture, minibatches, type='prodlda',learning_rate=0.01,
          batch_size=batch_size, training_epochs=100, display_step=5)

Initial Learning Rate: 0.01
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("Softmax_1:0", shape=(256, 8001), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 895.774136745
The approximated perplexity for test set is:  7046.7983
Epoch: 0006 cost= 672.931849190
The approximated perplexity for test set is:  3400.3796
Epoch: 0011 cost= 652.995368923
The approximated perplexity for test set is:  1892.3381
Epoch: 0016 cost= 624.165778635
The approximated perplexity for test set is:  1434.7699
Epoch: 0021 cost= 644.345398627
The approximated perplexity for test set is:  1184.8885
Epoch: 0026 cost= 604.180534553
The approximated perplexity for test set is:  1049.2427
Epoch: 0031 cost= 580.621092329
The approximated perplexity for test set is:  993.40393
Epoch: 0036 cost= 590.637583568
The approximated perplexity for test set is:  969.67804
Epoch: 0041 cost= 585.489200075
The approximated perplexity for tes

In [0]:
def top_words(beta, feature_names, n_top_words=10):
    res = []
    for i in range(len(beta)):
        res.append([feature_names[j] for j in beta[i].argsort()[:-n_top_words - 1:-1]])
    return res

In [0]:
topic_words = top_words(emb, list(zip(*sorted(token2id.items(), key=lambda x: x[1])))[0])

In [0]:
topic_words

[['dos',
  'windows',
  'card',
  'scsi',
  'apple',
  'printer',
  'duo',
  'cpu',
  'keyboard',
  'cmos'],
 ['thanks',
  'anyone_know',
  'thanks_advance',
  'hi',
  'windows',
  'helmet',
  'gordon_banks',
  'sale',
  'surrender_soon',
  'skepticism_chastity'],
 ['jumper',
  'adaptec',
  'floppy_disk',
  'dos',
  'hard_disk',
  'drives',
  'bios',
  'tape_drive',
  'jumpers',
  'nec'],
 ['atheists',
  'atheism',
  'argument',
  'militia',
  'atheist',
  'god',
  'conclusion',
  'religious',
  'must',
  'proposition'],
 ['flyers', 'nhl', 'unk', 'ahl', 'sabres', 'puck', 'que', 'pit', 'nyi', 'sj'],
 ['chip',
  'key_escrow',
  'clipper_chip',
  'cpsr',
  'serial_number',
  'nist',
  'private_sector',
  'astronaut',
  'encryption',
  'government'],
 ['wiring',
  'outlets',
  'wire',
  'infections',
  'illness',
  'public_health',
  'q',
  'health',
  'circuits',
  'aids'],
 ['gordon_banks',
  'skepticism_chastity',
  'surrender_soon',
  'intellect_shameful',
  'anyone_know',
  'thanks_ad

In [0]:
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(topics=topic_words, texts = data_test,dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print(coherence)

0.518
